IMPORTS, LOADING, AND PREPROCESSING

In [1]:
# import libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader

In [2]:
# check PyTorch and GPU setup
print("PyTorch Version:", torch.__version__)
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

PyTorch Version: 2.5.1+cu124
CUDA Available: True
GPU Name: NVIDIA L4


In [3]:
# preprocessing transformation
# convert images to PyTorch tensors
transform = transforms.Compose([transforms.ToTensor(),
                                ])

In [5]:
# load cifar-10 dataset
# partition into train and test sets
trainset=torchvision.datasets.CIFAR10(
    root='./data',
    train=True, # load training set (50k images)
    download=True,
    transform=transform
)

testset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False, # load test set (10k images)
    download=True,
    transform=transform
)

100%|██████████| 170M/170M [00:13<00:00, 12.9MB/s]


Extracting ./data/cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
# create dataloaders
train_loader = DataLoader(trainset, batch_size=4, shuffle=True)
test_loader = DataLoader(testset, batch_size=4, shuffle=False)

In [7]:
# inspect data
dataiter = iter(train_loader) # iterator to grab batch
images, labels = next(dataiter) # get first batch
print("Batch of images shape:", images.shape)
print("Batch of labels shape:", labels.shape)
print("Image value range:", images.min().item(), "to", images.max().item())

Batch of images shape: torch.Size([4, 3, 32, 32])
Batch of labels shape: torch.Size([4])
Image value range: 0.0 to 0.9215686321258545


In [8]:
# define class names for reference
class_names = ['airplane', 'automobile', 'bird', 'cat', 'deer',
               'dog', 'frog', 'horse', 'ship', 'truck']


print("Class names:", class_names)

Class names: ['airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
